In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np

2022-10-05 19:30:48.451788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 19:30:48.451965: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
file_path = 'datasets/heart.csv'
heart_data = pd.read_csv(file_path)

In [6]:
heart_data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

proc_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

In [9]:
heart_data_proc = proc_pipeline.fit_transform(heart_data)

In [10]:
heart_data_t = heart_data.drop("output", axis=1)

In [11]:
heart_data_z = imputer.fit_transform(heart_data_t)

In [12]:
heart_data_z

array([[63.,  1.,  3., ...,  0.,  0.,  1.],
       [37.,  1.,  2., ...,  0.,  0.,  2.],
       [41.,  0.,  1., ...,  2.,  0.,  2.],
       ...,
       [68.,  1.,  0., ...,  1.,  2.,  3.],
       [57.,  1.,  0., ...,  1.,  1.,  3.],
       [57.,  0.,  1., ...,  1.,  1.,  2.]])

In [13]:
heart_data_t

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [14]:
train_set, test_set = train_test_split(heart_data, test_size=0.2, random_state=42)
#heart_data_test = heart_data_proc.drop("output", axis=1)

In [15]:
X_train_full, X_test, y_train_full, y_test = train_test_split(heart_data_z, heart_data.output, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

'''scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)'''

'scaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_valid = scaler.transform(X_valid)\nX_test = scaler.transform(X_test)'

In [16]:
X_train_full.shape[1:]

(13,)

In [17]:
 X_test[0]

array([ 57. ,   1. ,   0. , 150. , 276. ,   0. ,   0. , 112. ,   1. ,
         0.6,   1. ,   1. ,   1. ])

In [33]:
class Model(tf.Module):
    def __init__(self):
        input = tf.keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = tf.keras.layers.Dense(50, activation="sigmoid")(input)
        hidden2 = tf.keras.layers.Dense(50, activation="sigmoid")(hidden1)
        #concat = keras.layers.concatenate([input, hidden2])
        output = tf.keras.layers.Dense(1)(hidden2)
        self.model = keras.models.Model(inputs=[input], outputs=[output])
        opt = tf.keras.optimizers.SGD(learning_rate=0.01)
        self.model.compile(loss="mse", optimizer=opt, metrics=['accuracy'])
        self.model.fit(X_train, y_train, epochs=20,
                       validation_data=(X_valid, y_valid))

    @tf.function(input_signature=[
        tf.TensorSpec([None, 13], tf.float32),
        tf.TensorSpec([None, ], tf.float32),
    ])
    def train(self, x, y):
        '''with tf.GradientTape() as tape:
            prediction = self.model(x)
            lossy = self.model.loss(y, prediction)
        gradients = tape.gradient(lossy, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))
        result = {"loss": lossy}
        return result'''

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None,13], dtype=tf.float64)
    ])
    def pred(self,x):
        return self.model(X_test[0])

    @tf.function(input_signature=[
        tf.TensorSpec([None, 13], tf.float32),
    ])
    def predictee(self, x):
        predictions = self.model(x)
        return {
            "rent": predictions
        }

    @tf.function(input_signature=[
        tf.TensorSpec([None, 13], tf.float32),
    ])
    def infer(self, x):
        logits = self.model(x)
        probabilities = tf.nn.softmax(logits, axis=-1)
        return {
            "output": probabilities,
            "logits": logits
        }

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def save(self, checkpoint_path):
        tensor_names = [weight.name for weight in self.model.weights]
        tensors_to_save = [weight.read_value() for weight in self.model.weights]
        tf.raw_ops.Save(
            filename=checkpoint_path, tensor_names=tensor_names,
            data=tensors_to_save, name='save')
        return {
            "checkpoint_path": checkpoint_path
        }

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def restore(self, checkpoint_path):
        restored_tensors = {}
        for var in self.model.weights:
            restored = tf.raw_ops.Restore(
                file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype, name='restore')
            var.assign(restored)
            restored_tensors[var.name] = restored
        return restored_tensors

In [34]:
m = Model()

Epoch 1/20
6/6 [==============================] - 1s 91ms/step - loss: 0.3856 - accuracy: 0.4706 - val_loss: 0.2392 - val_accuracy: 0.5614
Epoch 2/20
6/6 [==============================] - 0s 14ms/step - loss: 0.2363 - accuracy: 0.5765 - val_loss: 0.2374 - val_accuracy: 0.6491
Epoch 3/20
6/6 [==============================] - 0s 18ms/step - loss: 0.2349 - accuracy: 0.6647 - val_loss: 0.2343 - val_accuracy: 0.5614
Epoch 4/20
6/6 [==============================] - 0s 20ms/step - loss: 0.2400 - accuracy: 0.5353 - val_loss: 0.2333 - val_accuracy: 0.6140
Epoch 5/20
6/6 [==============================] - 0s 14ms/step - loss: 0.2341 - accuracy: 0.6412 - val_loss: 0.2342 - val_accuracy: 0.6316
Epoch 6/20
6/6 [==============================] - 0s 16ms/step - loss: 0.2331 - accuracy: 0.6353 - val_loss: 0.2441 - val_accuracy: 0.5789
Epoch 7/20
6/6 [==============================] - 0s 16ms/step - loss: 0.2382 - accuracy: 0.5941 - val_loss: 0.2375 - val_accuracy: 0.5614
Epoch 8/20
6/6 [===========

In [35]:
SAVED_MODEL_DIR = "saved_model"
tf.saved_model.save(
    m,
    SAVED_MODEL_DIR,
    signatures={
        'predictee':
            m.predictee.get_concrete_function(),
        'save':
            m.save.get_concrete_function(),
        'restore':
            m.restore.get_concrete_function(),
    })

ValueError: in user code:

    File "/tmp/ipykernel_85254/3137912130.py", line 32, in pred  *
        return self.model(X_test[0])
    File "/home/incognito/Anaconda/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/incognito/Anaconda/lib/python3.8/site-packages/keras/engine/input_spec.py", line 227, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" '

    ValueError: Exception encountered when calling layer "model_4" (type Functional).
    
    Input 0 of layer "dense_12" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (13,)
    
    Call arguments received:
      • inputs=tf.Tensor(shape=(13,), dtype=float64)
      • training=None
      • mask=None


In [38]:
import os

converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
converter.experimental_enable_resource_variables = True
tflite_model = converter.convert()

model_file_path = os.path.join('housing_model.tflite')
with open(model_file_path, 'wb') as model_file:
    model_file.write(tflite_model)

2022-10-05 20:02:03.767336: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-10-05 20:02:03.767379: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-10-05 20:02:03.767393: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-10-05 20:02:03.767950: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: saved_model
2022-10-05 20:02:03.772239: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-10-05 20:02:03.772283: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: saved_model
2022-10-05 20:02:03.784477: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-10-05 20:02:03.850335: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: saved_model
2022-10-05 20:02: